In [1]:
import pandas as pd
import datasets

from datasets import Dataset, load_dataset
from knowledge_graph_maker import Neo4jGraphModel
from generate_graph import get_propositions, generate
from main import query_graph, refine_query

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable


2024-10-23 13:33:19.816 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-23 13:33:19.847 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-23 13:33:19.847 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-23 13:33:19.847 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-10-23 13:33:19.847 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-23 13:33:19.848 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

### WikiQA

In [2]:
splits = {'test': 'data/test-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'train': 'data/train-00000-of-00001.parquet'}
df_wikiqa = pd.read_parquet("hf://datasets/microsoft/wiki_qa/" + splits["test"])

In [ ]:
df_wikiqa.info()

In [ ]:
df_wikiqa.sample(10)

### SQuAD

In [3]:
splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'validation': 'plain_text/validation-00000-of-00001.parquet'}
df_squad = pd.read_parquet("hf://datasets/rajpurkar/squad/" + splits["train"])

### NarrativeQA

In [ ]:
ds_narrativeqa = load_dataset("deepmind/narrativeqa")
ds_narrativeqa

In [ ]:
x = ds_narrativeqa['test']
type(x)

In [15]:
df = pd.DataFrame(x)

In [ ]:
# print(df.iloc[0]['document']['text'])
# print(df.iloc[0]['document']['summary']['text'])
# print(df.iloc[0]['question']['text'])
# print(df.iloc[0]['answers'][0]['text'])

# essay = df.iloc[0]['document']['text']

### QASPER

In [30]:
ds_qasper = load_dataset("allenai/qasper")

In [ ]:
ds_qasper

In [33]:
qasper_df = ds_qasper['test']

In [36]:
qasper_df = pd.DataFrame(qasper_df)

In [ ]:
qasper_df.info()

In [ ]:
qasper_df.head()

In [ ]:
qasper_df.iloc[0]['title']


In [ ]:
qasper_df.iloc[0]['full_text']['paragraphs']

In [ ]:
qasper_df.iloc[0]['qas']

### Cinderella

In [ ]:
with open('./data_input/txt/cinderella.txt') as file:
    essay = file.read()

### TEST PIPELINE

In [10]:
df_squad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        87599 non-null  object
 1   title     87599 non-null  object
 2   context   87599 non-null  object
 3   question  87599 non-null  object
 4   answers   87599 non-null  object
dtypes: object(5)
memory usage: 3.3+ MB


In [11]:
context_list = df_squad['context'].unique()

In [17]:
context_list[2:4]

array(['The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.',
       'The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five d

In [ ]:
# text = context_list[0:1][0].split(".")
# text

In [18]:
propositions = []

for context in context_list[2:4]:
    print(context)
    get_propositions(context, propositions)

The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.


/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


t: The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome)
t:  Its main seminary, Moreau Seminary, is located on the campus across St
t:  Joseph lake from the Main Building
t:  Old College, the oldest building on campus and located near the shore of St
t:  Mary lake, houses undergraduate seminarians
t:  Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto
t:  The university through the Moreau Seminary has ties to theologian Frederick Buechner
t:  While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching
The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes f

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


t: The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s
t:  Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B
t: S
t:  degrees offered
t:  Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B
t: A
t:  and Master of Business Administration (MBA) degrees, respectively


In [19]:
len(propositions)

45

In [20]:
propositions

['The university is the major seat of the Congregation of Holy Cross.',
 'The official headquarters of the Congregation of Holy Cross are in Rome.',
 'The main seminary is Moreau Seminary.',
 'Moreau Seminary is located on the campus across St.',
 'Joseph Lake is located from the Main Building.',
 'Old College is the oldest building on campus.',
 'Old College is located near the shore of St.',
 'Mary Lake is a location.',
 'Mary Lake houses undergraduate seminarians.',
 'Retired priests reside in Fatima House.',
 'Fatima House is a former retreat center.',
 'Retired brothers reside in Fatima House.',
 'Retired priests reside in Holy Cross House.',
 'Retired brothers reside in Holy Cross House.',
 'Retired priests reside in Columba Hall.',
 'Retired brothers reside in Columba Hall.',
 'Columba Hall is located near the Grotto.',
 'The university has ties to Frederick Buechner.',
 'Frederick Buechner is a theologian.',
 'The ties to Frederick Buechner are established through the Moreau Se

In [21]:
if generate(propositions):
    print("Success")
else:
    print("Failed")


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:39 - INFO 
Document: 1


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:39 - INFO 
Document Text: The university is the major seat of the Congregation of Holy Cross.


▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:49:39 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun'}, {'Object': "Objects are inanimate things that a person uses, Do not add the definite article 'the' in the object name"}, {'Event': 'An entity that happens at a specific time and place'}, {'Place': 'Places are locations where specific events took place and where persons can go to and where objects can be found'}, {'Miscellaneous': 'Any important concept can not be categorised with any other given label'}] relationships=['Relation between any pair of Entities']



Using Model:  gpt-4o-mini



▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:49:41 - INFO 
LLM Response:
[
   {
       "node_1": {
           "label": "Place",
           "name": "university"
       },
       "node_2": {
           "label": "Miscellaneous",
           "name": "Congregation of Holy Cross"
       },
       "relationship": "The university serves as the major seat for the Congregation of Holy Cross."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:41 - INFO 
Trying JSON Parsing: 
[
   {
       "node_1": {
           "label": "Place",
           "name": "university"
       },
       "node_2": {
           "label": "Miscellaneous",
           "name": "Congregation of Holy Cross"
       },
       "relationship": "The university serves as the major seat for the Congregation of Holy Cross."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:41 - INFO 
JSON Parsing Successful!


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:41 - INFO 
Document: 2


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:41 - INFO 
Document Text: The official hea

Using Model:  gpt-4o-mini



▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:49:42 - INFO 
LLM Response:
[
   {
       "node_1": {"label": "Place", "name": "Rome"},
       "node_2": {"label": "Miscellaneous", "name": "Congregation of Holy Cross"},
       "relationship": "The Congregation of Holy Cross has its official headquarters located in Rome."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:42 - INFO 
Trying JSON Parsing: 
[
   {
       "node_1": {"label": "Place", "name": "Rome"},
       "node_2": {"label": "Miscellaneous", "name": "Congregation of Holy Cross"},
       "relationship": "The Congregation of Holy Cross has its official headquarters located in Rome."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:42 - INFO 
JSON Parsing Successful!


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:42 - INFO 
Document: 3


▶︎ GRAPH MAKER LOG - 2024-10-23 12:49:42 - INFO 
Document Text: The main seminary is Moreau Seminary.


▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:49:42 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without

...

Using Model:  gpt-4o-mini



▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:50:48 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Miscellaneous",
            "name": "Master of Business Administration"
        },
        "node_2": {
            "label": "Miscellaneous",
            "name": "academic degree"
        },
        "relationship": "Master of Business Administration is a type of academic degree."
    }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:48 - INFO 
Trying JSON Parsing: 
[
    {
        "node_1": {
            "label": "Miscellaneous",
            "name": "Master of Business Administration"
        },
        "node_2": {
            "label": "Miscellaneous",
            "name": "academic degree"
        },
        "relationship": "Master of Business Administration is a type of academic degree."
    }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:48 - INFO 
JSON Parsing Successful!


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:48 - INFO 
Document: 45


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:48 

Using Model:  gpt-4o-mini



▶︎ GRAPH MAKER VERBOSE - 2024-10-23 12:50:49 - INFO 
LLM Response:
[
   {
       "node_1": {
           "label": "Miscellaneous",
           "name": "MBA"
       },
       "node_2": {
           "label": "Miscellaneous",
           "name": "Master of Business Administration"
       },
       "relationship": "MBA is an abbreviation for Master of Business Administration."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:49 - INFO 
Trying JSON Parsing: 
[
   {
       "node_1": {
           "label": "Miscellaneous",
           "name": "MBA"
       },
       "node_2": {
           "label": "Miscellaneous",
           "name": "Master of Business Administration"
       },
       "relationship": "MBA is an abbreviation for Master of Business Administration."
   }
]


▶︎ GRAPH MAKER LOG - 2024-10-23 12:50:49 - INFO 
JSON Parsing Successful!



Success


In [24]:
df_squad_reduced = df_squad[0:20]
df_squad_reduced

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
6,5733bf84d058e614000b61bf,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,"{'text': ['twice'], 'answer_start': [441]}"
7,5733bf84d058e614000b61c0,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,"{'text': ['The Observer'], 'answer_start': [598]}"
8,5733bf84d058e614000b61bd,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,"{'text': ['three'], 'answer_start': [126]}"
9,5733bf84d058e614000b61c1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,"{'text': ['1987'], 'answer_start': [908]}"


In [27]:
# Define a function to apply to each row
def print_qa(row):
    
    question = row['question']
    print("Question:", question)

    result = query_graph(question)
    model_answer = result['result']
    real_answer = row['answers']['text'][0]

    cypher_query = result['intermediate_steps'][0]['query']
    context = result['intermediate_steps'][1]['context']
    
    if(len(context) < 1):
        result = refine_query(cypher_query[6:], question)
        model_answer = result['result'] 

    f = open("results.txt", "a")
   

    f.write("Question: " + question)
    f.write("\n")
    f.write("Model Answer: " + model_answer)
    f.write("\n")
    f.write("Real Answer: " + real_answer)
    f.write("\n")
    f.write("\n")

    f.close()
    # print("Question:", question)
    # print("Model Answer:", model_answer)
    # print("Real Answer:", real_answer)
    # print("=========================================================================================================")
    
    

In [28]:
# Apply the function to each row
df_squad_reduced.apply(print_qa, axis=1)

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]-(re:Entity)
WHERE toLower(e.name) =~ '.*\\b(virgin|mary|lourdes|france)\\b.*' 
AND toLower(r.description) CONTAINS '1858'
RETURN e.name, r.metadata, r.description, re.name

Full Context:
[{'e.name': 'appearance of the Virgin Mary', 'r.metadata': '{"summary": "The appearance of the Virgin Mary to Saint Bernadette Soubirous occurred in 1858.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'The appearance of the Virgin Mary occurred in 1858.', 're.name': '1858'}]

> Finished chain.
Question: What is in front of the Notre Dame Main Building?


> Entering new GraphCypherQAChain chain...


/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/neo4j/_sync/driver.py:511: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]-(re:Entity)
WHERE toLower(e.name) =~ '.*\\b(not|notre|dame|main|building)\\b.*'
RETURN re.name, r.metadata, r.description

Full Context:
[{'re.name': 'Joseph Lake', 'r.metadata': '{"summary": "Joseph Lake is located from the Main Building.", "generated_at": "2024-10-23 12:49:39.798741"}', 'r.description': 'Joseph Lake is located in relation to the Main Building.'}, {'re.name': 'Basilica of Sacred Heart', 'r.metadata': '{"summary": "The Basilica of the Sacred Heart is next to the Main Building.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'Basilica of Sacred Heart is located next to Main Building.'}, {'re.name': 'copper statue of Christ', 'r.metadata': '{"summary": "The copper statue of Christ is facing the Main Building.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'The copper statue of Christ is positioned in a way that it is facing the Main Building.'}, {'re.name': 'copper statue', 'r.me

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/neo4j/_sync/driver.py:511: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]-(re:Entity)
WHERE toLower(e.name) =~ '.*\\b(basilica|sacred|heart|notre|dame)\\b.*'
RETURN e.name, r.metadata, r.description, re.name

Full Context:
[{'e.name': 'Basilica of Sacred Heart', 'r.metadata': '{"summary": "The Basilica of the Sacred Heart is next to the Main Building.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'Basilica of Sacred Heart is located next to Main Building.', 're.name': 'Main Building'}, {'e.name': 'basilica', 'r.metadata': '{"summary": "The Grotto is located immediately behind the basilica.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'Grotto is located immediately behind basilica.', 're.name': 'Grotto'}, {'e.name': 'Notre Dame', 'r.metadata': '{"summary": "Buechner has praised writers from Notre Dame.", "generated_at": "2024-10-23 12:49:39.798741"}', 'r.description': 'Buechner has praised writers associated with Notre Dame.', 're.name': 'Buechner'}, {'e.name': 'N

...

Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]-(re:Entity)
WHERE toLower(e.name) =~ '.*\\b(stinson|remick|hall|engineering)\\b.*'
RETURN e.name, r.metadata, r.description, re.name

Full Context:
[{'e.name': 'Columba Hall', 'r.metadata': '{"summary": "Columba Hall is located near the Grotto.", "generated_at": "2024-10-23 12:49:39.798741"}', 'r.description': 'Columba Hall is located near the Grotto.', 're.name': 'Grotto'}, {'e.name': 'Columba Hall', 'r.metadata': '{"summary": "Retired brothers reside in Columba Hall.", "generated_at": "2024-10-23 12:49:39.798741"}', 'r.description': 'The brothers reside in Columba Hall.', 're.name': 'brothers'}, {'e.name': 'Columba Hall', 'r.metadata': '{"summary": "Retired priests reside in Columba Hall.", "generated_at": "2024-10-23 12:49:39.798741"}', 'r.description': 'Retired priests live in Columba Hall.', 're.name': 'Retired priests'}, {'e.name': 'College of Engineering', 'r.metadata': '{"summary": "The College of Engineering was established

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/neo4j/_sync/driver.py:511: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


Generated Cypher:
cypher
MATCH (e:Entity)-[r:RELATED]-(re:Entity)
WHERE toLower(e.name) =~ '.*\\b(college|of|science|civil|engineering|courses|notre|dame)\\b.*'
RETURN e.name, r.metadata, r.description, re.name

Full Context:
[{'e.name': 'copper statue of Christ', 'r.metadata': '{"summary": "The copper statue of Christ is facing the Main Building.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'The copper statue of Christ is positioned in a way that it is facing the Main Building.', 're.name': 'Main Building'}, {'e.name': 'Basilica of Sacred Heart', 'r.metadata': '{"summary": "The Basilica of the Sacred Heart is next to the Main Building.", "generated_at": "2024-10-22 20:25:12.768706"}', 'r.description': 'Basilica of Sacred Heart is located next to Main Building.', 're.name': 'Main Building'}, {'e.name': 'Marian place of prayer and reflection', 'r.metadata': '{"summary": "The Grotto is a Marian place of prayer and reflection.", "generated_at": "2024-10-22 20:25:12.7

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/neo4j/_sync/driver.py:511: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object